In [ ]:
# default_exp tex

# TeX

> A collection of functions to ease the inclusion of $\TeX$ strings in Python source code.

In [ ]:
# hide
# to avoid tqdm's experimental warning
import warnings
warnings.filterwarnings("ignore", message='Using `tqdm.autonotebook.tqdm` in notebook mode')

from py2gift.util import render_latex

In [ ]:
# export
import functools
from typing import Union, Iterable, List, Optional, Tuple

import numpy as np

## Decorating a function so that it can return a formula

A decorator that allows (optionally) the string returned by any function to be enclosed between `$`s.

In [ ]:
# export

def to_formula_maybe(func):
    
    def wrapper(*args, **kwargs):
        
        if ('to_formula' in kwargs) and (kwargs['to_formula']):
            
            kwargs.pop('to_formula')
            
            return f'${func(*args, **kwargs)}$'
        
        else:
            
            if ('to_formula' in kwargs):
            
                # it must also be popped out
                kwargs.pop('to_formula')
            
            return func(*args, **kwargs)
        
    functools.update_wrapper(wrapper, func)
        
    return wrapper

In [ ]:
def to_radians(n_cycles: int) -> float:
    
    return n_cycles * 3.14

In [ ]:
to_radians(2)

6.28

A module that allows to inspect functions.

In [ ]:
import inspect

In [ ]:
inspect.signature(to_radians)

<Signature (n_cycles: int) -> float>

In [ ]:
@to_formula_maybe
def to_radians(n_cycles: int) -> float:
    
    return n_cycles * 3.14

In [ ]:
inspect.signature(to_radians)

<Signature (n_cycles: int) -> float>

In [ ]:
to_radians(2)

6.28

In [ ]:
to_radians(2, to_formula=True)

'$6.28$'

`render_latex` can be used to properly render it in the notebook:

In [ ]:
render_latex(to_radians(2, to_formula=True))

$\Large 6.28$

# Convenience functions

## Enumerating the strings in a list

In [ ]:
# export
def join(strings_list: List[str], nexus: str = 'and', to_formula: bool = True):
    """
    Enumerates the strings in a list, optionally enclosing every element between `$`s.
    
    **Parameters**
    
    - `strings_list`: list
        
        A list with the strings to be joined.
    
    - `nexus`: str
        
        Text between the second to last and last elements.
    
    - `to_formula`: bool, optional
        
        If True every string will be enclosed in '$'s.

    **Returns**

    - `out`: str
        
        TeX compatible string.
    """
    
    if to_formula:
        
        delimiter = '$'
    
    else:
        
        delimiter = ''
    
    return delimiter + f'{delimiter}, {delimiter}'.join(strings_list[:-1]) + f'{delimiter} {nexus} {delimiter}{strings_list[-1]}{delimiter}'

In [ ]:
render_latex(join(['a', 'b', 'c']))

$\Large a$, $\Large b$ and $\Large c$

In [ ]:
render_latex(join(['a', 'b', 'c'], nexus='y'))

$\Large a$, $\Large b$ y $\Large c$

In [ ]:
render_latex(join(['2', '\sigma']))

$\Large 2$ and $\Large \sigma$

# Definitions of functions

*Customizable* mathematical functions.

## Gaussian probability density function

The expression of a (univariate) Gaussian probability density function.

In [ ]:
# export
@to_formula_maybe
def gaussian_pdf(x: str = 'x', mean: str = r'\mu', variance: str = r'\sigma^2') -> str:
    """
    Returns a string representing the probability density function for a Gaussian distribution.
    
    **Parameters**
    
    - `x`: str
        
        The random variable.
    
    - `mean`: str, optional
        
        The mean of the random variable.
    
    - `variance`: str, optional
    
        The variance of the random variable.

    **Returns**
    
    - `out`: str
        
        TeX compatible string.
    """

    return r'\frac{1}{\sqrt{2\pi ' + variance + r'}}e^{-\frac{(' + x + '-' + mean + r')^2}{2' + variance + r'}}'

With no arguments, it yields the usual formula,

In [ ]:
render_latex(gaussian_pdf(to_formula=True))

$\Large \frac{1}{\sqrt{2\pi \sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

The arguments allow to use different symbols for the random variable, the mean and the variance

In [ ]:
render_latex(gaussian_pdf(x="n", mean="m", variance="v", to_formula=True))

$\Large \frac{1}{\sqrt{2\pi v}}e^{-\frac{(n-m)^2}{2v}}$

## Stirling's approximation to the Q function

In [ ]:
# export
@to_formula_maybe
def q_function_approximation(x: str = 'x') -> str:
    """
    Returns a string representing the Stirling approximation for the Q function.
    
    **Parameters**
    
    - `x`: str
        
        The argument of the Q function.

    **Returns**
    
    `out`: str
        
        TeX compatible string.
    """

    return f'Q({x}) \\approx \\frac{{1}}{{2}} e^{{-\\frac{{{x}^2}}{{2}}}}'

With no arguments, the argument is $x$

In [ ]:
render_latex(q_function_approximation(to_formula=True))

$\Large Q(x) \approx \frac{1}{2} e^{-\frac{x^2}{2}}$

but a specific variable can be passed

In [ ]:
render_latex(q_function_approximation('t', to_formula=True))

$\Large Q(t) \approx \frac{1}{2} e^{-\frac{t^2}{2}}$

## Part-wise defined functions

In [ ]:
# export
@to_formula_maybe
def partwise_function(function: str, parts: List[Tuple[str, str]], add_zero_otherwise: bool = True) -> str:
    """
    Returns a string representing the definition a part-wise mathematical function.
    
    **Parameters**
    
    - `function`: str
        
        The name of the function.
    - `parts`: list
        
        Each element is a tuple yields whose 1st element is the value of the function and whose second is a condition stating where the 1st applies.
    - `add_zero_otherwise`: bool
        
        If True, one last part stating "0, otherwise" is added.

    **Returns**
    
    `out`: str
        TeX compatible string.
    """
    
    res = f'{function}='
    
    res += '\\begin{cases}\n'
    
    for p in parts:
    
        res += f'{p[0]},& {p[1]} \\\\'
    
    if add_zero_otherwise:
        
        res += r'0,& \text{otherwise}'
    
    res += r'\end{cases}'
    
    return res

In [ ]:
tex = partwise_function('f(x)', [('x+1', '0 < x \le 1'), ('x-1', '1 < x \le 2')], to_formula=True)
render_latex(tex)

$\Large f(x)=\begin{cases}
x+1,& 0 < x \le 1 \\x-1,& 1 < x \le 2 \\0,& \text{otherwise}\end{cases}$

In [ ]:
tex = partwise_function('f(x)', [('x+1', '0 < x \le 1')], add_zero_otherwise=False, to_formula=True)
render_latex(tex)

$\Large f(x)=\begin{cases}
x+1,& 0 < x \le 1 \\\end{cases}$

# Numbers

Utilities to turn numbers (or collections thereof) into $\LaTeX$ source code.

## Scalar

In [ ]:
# export
@to_formula_maybe
def from_number(n: Union[int, float], prefix: str = '', precision: int = 3, fixed_point_format: bool = False) -> str:
    """
    Returns a string for a given number.
    
    Parameters
    ----------
    n: int or float
        The number.
    prefix: str
        A string to be prepended to the number.
    precision: int
        Number of decimals (ignored if the number is an integer).
    fixed_point_format: bool
        If True, a fixed-point format (f) is used regardless of the actual type.
    
        

    Returns
    -------
    out: str
        TeX compatible string.
    """
    
    format_specifier = f'.{precision}{"f" if fixed_point_format else "g"}'

    return f'{prefix}{n:{format_specifier}}'

Valid for both floating point numbers,

In [ ]:
tex = from_number(2.3, to_formula=True)
render_latex(tex)

$\Large 2.3$

and integers

In [ ]:
tex = from_number(3, to_formula=True)
render_latex(tex)

$\Large 3$

With a higher `precision`

In [ ]:
tex = from_number(np.pi, precision=6, to_formula=True)
render_latex(tex)

$\Large 3.14159$

If the number is an integer, `precision` parameter is ignored

In [ ]:
tex = from_number(42, to_formula=True)
render_latex(tex)

$\Large 42$

A prefix can be passed:

In [ ]:
tex = from_number(42, prefix='a=', to_formula=True)
render_latex(tex)

$\Large a=42$

## Matrix or vector/list

Functions to turn a matrix or vector into $\LaTeX$ source code.

In [ ]:
# export
@to_formula_maybe
def from_matrix(m: Union[list, np.ndarray], float_point_precision: int = 3) -> str:
    """
    Returns a string for a given array or matrix.
    
    Parameters
    ----------
    m: list or ndarray
        A numpy array or a list.
    float_point_precision: int
        Number of decimals (ignored if the number is an integer).

    Returns
    -------
    out: str
        TeX compatible string.
    """
    
    format_from_number = lambda x: f'.{float_point_precision}g' if (type(x) == np.float64) or (type(x) == float) else f'd'

    if isinstance(m[0], (list, np.ndarray)):

        return r'\begin{bmatrix}' + r' \\ '.join(
            [(r' & '.join([f'{e:{format_from_number(m[0][0])}}' for e in row])) for row in m]) + r'\end{bmatrix}'

    else:
        
        return r'\begin{bmatrix}' + r' & '.join([f'{e:{format_from_number(m[0])}}' for e in m]) + r'\end{bmatrix}'

`from_matrix` returns $\LaTeX$ source code for a list or `np.ndarray`

This can be applied on lists

In [ ]:
render_latex(from_matrix([1, 2, 3], to_formula=True) + ',')

$\Large \begin{bmatrix}1 & 2 & 3\end{bmatrix}$,

or numpy arrays, both *2D*

In [ ]:
render_latex(from_matrix(np.array([[1.11, 3.14], [14.2, 5.1]]), to_formula=True) + ',')

$\Large \begin{bmatrix}1.11 & 3.14 \\ 14.2 & 5.1\end{bmatrix}$,

and *1D*

In [ ]:
render_latex(from_matrix(np.array([14.2, 5.1]), to_formula=True))

$\Large \begin{bmatrix}14.2 & 5.1\end{bmatrix}$

If the numbers are integers, that is taken into account:

In [ ]:
render_latex(from_matrix(np.array([[1, 3], [4, 5]]), to_formula=True))

$\Large \begin{bmatrix}1 & 3 \\ 4 & 5\end{bmatrix}$

In [ ]:
render_latex(from_matrix(np.array([14, 5]), to_formula=True))

$\Large \begin{bmatrix}14 & 5\end{bmatrix}$

In [ ]:
render_latex(from_matrix([1, 2], to_formula=True))

$\Large \begin{bmatrix}1 & 2\end{bmatrix}$

# Dot product

In [ ]:
# export
@to_formula_maybe
def dot_product(
    lhs_template: str, lhs: list, rhs_template: str, rhs: list, product_operator: str = '',
    addition_operator: str = '+') -> str:
    """
    Returns a string for the dot product of two vectors, regardless of whether they are symbols or numbers.
    
    Parameters
    ----------
    lhs_template: str
        Left-hand side template; it should include a replacement field ({}) that will be replaced by one of
        the elements in `lhs`
    lhs: list
        Left-hand side elements.
    rhs_template: str
        Right-hand side template; it should include a replacement field ({}) that will be replaced by one of
        the elements in `rhs`
    rhs: list
        Right-hand side elements.
    product_operator: str
        Symbol to be used as product operator.
    addition_operator: str
        Symbol to be used as addition operator.

    Returns
    -------
    out: str
        TeX compatible string.
    """
    
    return addition_operator.join([lhs_template.format(l) + product_operator + rhs_template.format(r) for l,r in zip(lhs, rhs)])

In [ ]:
render_latex(dot_product('p(y_1|x_{})', [1, 2], 'p(x_{})', [1,2], to_formula=True))

$\Large p(y_1|x_1)p(x_1)+p(y_1|x_2)p(x_2)$

In [ ]:
render_latex(dot_product('{}', [0.1, 0.2], '{}', [0.75, 0.9], product_operator=r'\cdot', to_formula=True))

$\Large 0.1\cdot0.75+0.2\cdot0.9$

# Law of total probability

In [ ]:
# export
@to_formula_maybe
def total_probability(fixed_symbol: str, varying_symbol_template: str, n: int, start_at: int = 1) -> str:
    
    return '+'.join([f'p({fixed_symbol},{varying_symbol_template.format(i)})' for i in range(start_at, start_at+n)])

In [ ]:
render_latex(total_probability('x_1', 'y_{}', 3, to_formula=True))

$\Large p(x_1,y_1)+p(x_1,y_2)+p(x_1,y_3)$

Starting at a different index

In [ ]:
render_latex(total_probability('x_1', 'y_{}', 3, start_at=3, to_formula=True))

$\Large p(x_1,y_3)+p(x_1,y_4)+p(x_1,y_5)$

# Enumerations

Given a list of `float`s,  `enumerate_math` builds a $\TeX$ string in which the value of each one is printed after (optionally) being assigned to an indexed variable that follows a given pattern.

* `numbers_list` is a list of `float`s
* `assigned_to` is a string with a [replacement field](https://docs.python.org/3/library/string.html#format-string-syntax). This means that any `{` or `}` must be escaped by doubling it.
* `precision` is the number of decimal places

In [ ]:
# export

def enumerate_math(
    numbers_list: List[float], assigned_to: Optional[str] = None, nexus: str = 'and', precision: int = 3, start_at: int = 1) -> str:
    """
    Returns a string for a enumeration of formulas.
    """

#     format_specifier = f'.{precision}f'
    format_specifier = f'.{precision}g'

    strings_list = [f'{e:{format_specifier}}' for e in numbers_list]

    if assigned_to:

        strings_list = [assigned_to.format(i_s) + ' = ' + s for i_s, s in enumerate(strings_list, start_at)]
    
    return join(strings_list, nexus=nexus)

Using the optional `assigned_to` arguement,

In [ ]:
render_latex(enumerate_math([0.7, 0.9], assigned_to='w_t^{{({})}}'))

$\Large w_t^{(1)} = 0.7$ and $\Large w_t^{(2)} = 0.9$

and without it

In [ ]:
render_latex(enumerate_math([0.7, 0.9]))

$\Large 0.7$ and $\Large 0.9$

Starting at a different index on the left-hand-side

In [ ]:
render_latex(enumerate_math([0.722341, 0.9], assigned_to='w_t^{{({})}}', start_at=2))

$\Large w_t^{(2)} = 0.722$ and $\Large w_t^{(3)} = 0.9$

---

`enumerate_assignments` constructs a enumeration of assignments from left-hand-side and right-hand-side templates and right-hand-side values. It's similar to `enumerate_math` when the argument `assigned_to` is passed to the latter, but more general since the right-hand expression is also obtained from a template.

* `lhs_template` is a string with a [replacement field](https://docs.python.org/3/library/string.html#format-string-syntax) that will be replaced by an **index**
* `rhs_template` is a string with a [replacement field](https://docs.python.org/3/library/string.html#format-string-syntax) that will be replaced by one of the corresponding elements in `rhs`
* `rhs` is a list of `float`s

In [ ]:
# export

def enumerate_assignments(
    lhs_template: str, rhs_template: str, rhs: List[float], nexus: str = 'and', precision: int = 3, start_at: int = 1) -> str:
    """
    Returns a string for a enumeration of formulas.
    """
    
    return join([f'{lhs_template} = {rhs_template}'.format(i, r) for i, r in enumerate(rhs, start_at)])

In [ ]:
render_latex(enumerate_assignments('s_{}', '{}', [1.3,4.1]))

$\Large s_1 = 1.3$ and $\Large s_2 = 4.1$

In [ ]:
render_latex(enumerate_assignments('s_{}', '2^{{{}}}', [1.3,4.1], start_at=4))

$\Large s_4 = 2^{1.3}$ and $\Large s_5 = 2^{4.1}$

---

In [ ]:
# export

def expand(template: str, n: int, to_math: bool = False, nexus: str = 'and', start_at: int = 1) -> str:
    """
    Expand a symbol according to a pattern.

    >>> util.expand('s_{}', 3, True)
    '$s_1$, $s_2$ and $s_3$'

    Parameters
    ----------
    template : str
        String with a *single* replacement field ({})
    n : int
        Requested number of terms
    to_math : bool
        If `True`, every output term is enclosed between $'s
    nexus : str
        String joining the second to last and last terms.
    start_at: int
        The number at which indexes start.

    Returns
    -------

    """
    
    return join([template.format(i) for i in range(start_at, start_at + n)], to_formula=to_math)

assert expand('s_{}', 3, True) == '$s_1$, $s_2$ and $s_3$'

In [ ]:
render_latex(expand('s_{}', 3, True))

$\Large s_1$, $\Large s_2$ and $\Large s_3$

In [ ]:
# hide

import nbdev.export
nbdev.export.notebook2script('01_tex.ipynb')

Converted 01_tex.ipynb.
